The following is nowhere near the extent of combinations of things I've tried in order to get this network to train. 
I have overwritten dozens of versions of for defining tranforms and datasets, defining model architecture, and defining the training/validation loops. 

I reset the kernel before each attempt to clear out existing variables (but I wonder if a problem may exist in that perhaps resetting the kernel might not be clearing the GPU memory?)

Results have been mixed, but never good. 

Sometimes I get a few training epochs that look hopeful, before the loss suddenly explodes to very high numbers. 
Sometimes I get what look like multiple epochs with tiny improvements in the loss, before it then begins bouncing around with no progress over time, always stuck in the range of 4.8...

I have found in the latter cirumstance, when I include code to print the top class probability, that the network is making the same prediction over and over for every photo in multiple batches (anywhere from 8 - 40 batches at a time with the same single class prediction for every image), before switching to a new class being predicted for the next 8 to 40 batches, with this staying consistent through the training epochs.

Test batches, then, of course, come out the same way - a single class predicted for all images in a batch.

**While my training loops here are limited to 10 epochs, this is only so I can get this notebook created and uploaded tonight. I have run all these and many many more versions of this code with 30-100+ epochs. If it's making no progress by 10 epochs, it makes no progress by 30 either.**

It has always been clear by the 10th epoch whether or not the training is progressing properly.
No version has ever achieved above 1% accuracy in testing (which is consistent with the behavior of making the same prediction for image after image)

# Latest Model Implementing Mentor Suggestions

### Transforms, datasets and dataloaders
**Training Data Transforms:** Random Horizontal Flip, RandomRotation(30), Resize images so smallest dimension is 250, then RandomCrop to 224x224. Converted to Tensor, then Normalized with the means and stds considered optimal.

**Validation and Testing Data Transforms:** Resized to smallest dimension 250, then Center Cropped to 224x224, converted to tensor, then normalized with optimal mean/std values

batch_size - set to 8

all datasets shuffled into DataLoaders

In [1]:
import os
import numpy as np
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

## Define transforms, Datasets, batch_size, and Dataloaders

#
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(30),
                                      transforms.Resize(250),
                                      transforms.RandomCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])
                                     ])

test_transform = transforms.Compose([transforms.Resize(250),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], 
                                                          [0.229, 0.224, 0.225])
                                    ])


data_dir = 'dogImages/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

print('Num Training imgs: ', len(train_data))
print('Num Validation imgs: ', len(valid_data))
print('Num Test imgs: ', len(test_data))

batch_size = 8
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)


loaders_scratch = {'train': train_loader, 
                   'valid': valid_loader, 
                   'test': test_loader}

classes = train_data.classes

Num Training imgs:  6680
Num Validation imgs:  835
Num Test imgs:  836


## Model Architecture
* 3 Convolutional Layers (all with kernel_size=3, padding=1), 3 Max Pooling Layers (kernel=2, stride=2), 2 Fully Connected Layers
* relu activation applied to each convolutional layer output, followed by batch normalization.
* 30% dropout probability between 1st and 2nd convolutional layers
* MaxPool layer of kernel size=2, stride=2 after each convolutional layer
* ReLU non-linearity applied to output of 1st fully connected layer. 
* No dropout in FC layers.

* Cross-Entropy Loss Function
* Adam optimizer with learning rate = 0.002

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.batchnorm4 = nn.BatchNorm2d(256)
#         self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
#         self.batchnorm5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128*28*28, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 133, bias=True)
#         self.fc3 = nn.Linear(500, 133, bias=True)
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.pool(x)
#         x = self.batchnorm4(F.relu(self.conv4(x)))
#         x = self.dropout(x)
#         x = self.pool(x)
#         x = self.batchnorm5(F.relu(self.conv5(x)))
#         x = self.pool(x)
        
        x = x.view(-1, 128*28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

In [3]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.0025)

### Training and Validation

all very much standard according to the way the models in the program to this point have been written.

validation loop inside `with torch.no_grad():`


In [4]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data) #get predictions
            loss = criterion(output, target) #calulate loss
            loss.backward()  # calculate the gradients
            optimizer.step() # perform optimization step
            train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

# train the model
model_scratch = train(25, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch_newest.pt')

Epoch: 1 	Training Loss: 10.118235 	Validation Loss: 4.906532
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.910345 	Validation Loss: 4.881038
Validation Loss Decreased. Saving model
Epoch: 3 	Training Loss: 4.885892 	Validation Loss: 4.869480
Validation Loss Decreased. Saving model
Epoch: 4 	Training Loss: 4.933945 	Validation Loss: 4.868146
Validation Loss Decreased. Saving model
Epoch: 5 	Training Loss: 4.866847 	Validation Loss: 4.867838
Validation Loss Decreased. Saving model
Epoch: 6 	Training Loss: 4.866725 	Validation Loss: 4.868748
Epoch: 7 	Training Loss: 4.866579 	Validation Loss: 4.870161
Epoch: 8 	Training Loss: 4.866401 	Validation Loss: 4.868974
Epoch: 9 	Training Loss: 4.866455 	Validation Loss: 4.869438
Epoch: 10 	Training Loss: 4.866441 	Validation Loss: 4.871444
Epoch: 11 	Training Loss: 4.866513 	Validation Loss: 4.869052
Epoch: 12 	Training Loss: 4.866430 	Validation Loss: 4.869843
Epoch: 13 	Training Loss: 4.866408 	Validation Loss: 4.870361
Ep

### Testing
using the code supplied by Udacity, but adding a couple print statements to view the predictions

In [5]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.0
    correct = 0.0
    total = 0.0
    model.eval()
    
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            print('Data: ', data)
            print('targets: ', target)
            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            print('Predictions: ', pred)
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Data:  tensor([[[[-0.1999, -0.3369, -0.3883,  ..., -0.1657, -0.1486, -0.1486],
          [-0.2171, -0.3369, -0.3712,  ..., -0.1999, -0.2684, -0.3541],
          [-0.2513, -0.3369, -0.3712,  ..., -0.1657, -0.2856, -0.4397],
          ...,
          [ 0.7591,  0.7762,  0.7933,  ...,  0.9817,  0.9474,  1.0502],
          [ 0.8618,  0.9132,  0.8789,  ...,  0.7591,  0.9303,  0.8961],
          [ 0.7762,  0.7762,  0.7419,  ...,  0.8104,  0.5878,  0.6392]],

         [[-0.9328, -1.0553, -1.0553,  ..., -0.8803, -0.8452, -0.8452],
          [-0.9678, -1.0553, -1.0553,  ..., -0.9153, -0.9678, -1.0553],
          [-0.9853, -1.0553, -1.0553,  ..., -0.8803, -0.9853, -1.1429],
          ...,
          [ 0.2227,  0.2577,  0.2577,  ...,  1.0980,  1.1155,  1.1681],
          [ 0.3102,  0.3803,  0.3452,  ...,  0.8880,  1.0980,  0.9930],
          [ 0.1001,  0.1176,  0.1176,  ...,  0.9405,  0.6954,  0.7129]],

         [[-1.4559, -1.4384, -1.3513,  ..., -1.4384, -1.3861, -1.3861],
          [-1.4907, -1.

Data:  tensor([[[[-0.4054,  0.3823,  0.5022,  ...,  0.7077,  1.2899,  0.3994],
          [-0.5424,  0.4508,  0.5022,  ...,  1.3070,  0.6392,  0.3652],
          [-0.4054,  0.2453,  0.3138,  ...,  0.9132,  0.4337,  0.3309],
          ...,
          [ 0.4337,  0.5707,  0.6563,  ...,  0.0398,  0.0741,  0.1597],
          [ 0.4679,  0.5878,  0.6049,  ...,  0.0741,  0.0741,  0.1426],
          [ 0.6049,  0.5707,  0.4337,  ...,  0.1939,  0.0912,  0.0912]],

         [[-0.2850,  0.5203,  0.6429,  ...,  0.5903,  1.1856,  0.2577],
          [-0.4251,  0.5903,  0.6429,  ...,  1.2206,  0.5203,  0.2227],
          [-0.2850,  0.3803,  0.4503,  ...,  0.7829,  0.2927,  0.1877],
          ...,
          [ 0.2577,  0.3978,  0.5203,  ...,  0.0126,  0.0476,  0.1352],
          [ 0.2927,  0.4153,  0.4503,  ...,  0.0476,  0.0476,  0.1176],
          [ 0.4328,  0.3978,  0.2927,  ...,  0.1702,  0.0651,  0.0651]],

         [[-0.0615,  0.7402,  0.8622,  ...,  0.5485,  1.1411,  0.2173],
          [-0.2010,  0.

Data:  tensor([[[[-0.5596, -0.3712, -0.1999,  ..., -1.4158, -1.4329, -1.4672],
          [-0.5767, -0.3883, -0.2513,  ..., -1.4329, -1.4329, -1.4500],
          [-0.5767, -0.4397, -0.3369,  ..., -1.3815, -1.4158, -1.4158],
          ...,
          [ 0.1768,  0.1939,  0.1597,  ...,  0.6392,  0.5878,  0.7419],
          [ 0.1768,  0.2796,  0.2111,  ...,  0.5364,  0.3481,  0.2453],
          [ 0.2624,  0.2624,  0.2111,  ...,  0.8789,  0.6392,  0.3481]],

         [[-0.2675, -0.0749,  0.0651,  ..., -1.1078, -1.1604, -1.1954],
          [-0.2500, -0.0924,  0.0126,  ..., -1.1779, -1.1954, -1.2654],
          [-0.2850, -0.1450, -0.0924,  ..., -1.1954, -1.2479, -1.2829],
          ...,
          [-0.1450, -0.1450, -0.1450,  ...,  0.8704,  0.8354,  0.9055],
          [-0.0574, -0.0049, -0.1099,  ...,  0.7654,  0.6254,  0.5378],
          [ 0.0476,  0.0301, -0.0924,  ...,  1.1856,  0.9055,  0.7304]],

         [[-0.6193, -0.3404, -0.2184,  ..., -1.2467, -1.2467, -1.2641],
          [-0.6890, -0.

Data:  tensor([[[[ 0.8789,  0.8961,  0.9646,  ...,  0.7248,  0.6734,  0.6049],
          [ 0.9132,  0.9303,  0.9132,  ...,  0.7419,  0.6392,  0.5536],
          [ 0.8961,  0.9132,  0.9303,  ...,  0.6221,  0.5878,  0.5022],
          ...,
          [ 1.1015,  1.1015,  1.1358,  ...,  1.6324,  1.7180,  1.9407],
          [ 1.1187,  1.1700,  1.1529,  ...,  1.6324,  1.7180,  1.7865],
          [ 1.1700,  1.1872,  1.1872,  ...,  1.5468,  1.4440,  1.5810]],

         [[ 1.0105,  1.0105,  1.0630,  ...,  0.8704,  0.8529,  0.7829],
          [ 1.0280,  1.0455,  1.0280,  ...,  0.8704,  0.7654,  0.6954],
          [ 1.0105,  1.0105,  1.0105,  ...,  0.7304,  0.6779,  0.6078],
          ...,
          [ 1.1155,  1.1331,  1.1681,  ...,  1.6758,  1.7808,  2.0434],
          [ 1.1681,  1.2206,  1.2031,  ...,  1.6758,  1.7633,  1.8508],
          [ 1.2556,  1.2731,  1.2731,  ...,  1.5707,  1.5007,  1.6583]],

         [[ 0.9494,  0.9668,  1.0365,  ...,  0.7576,  0.7402,  0.6356],
          [ 0.9668,  1.

Data:  tensor([[[[ 1.2728,  1.2557,  1.2385,  ..., -0.9705, -1.1075, -1.7412],
          [ 1.2728,  1.2728,  1.2728,  ..., -0.9534, -1.0733, -1.7069],
          [ 1.2899,  1.2899,  1.2728,  ..., -0.9363, -1.0562, -1.6042],
          ...,
          [ 0.4679,  0.4166,  0.3652,  ...,  0.3138,  0.3481,  0.3823],
          [ 0.4337,  0.4166,  0.3652,  ...,  0.3138,  0.3481,  0.3652],
          [ 0.4166,  0.3994,  0.3652,  ...,  0.2967,  0.3481,  0.3652]],

         [[ 1.2731,  1.2556,  1.2381,  ..., -0.9503, -1.0728, -1.6856],
          [ 1.2731,  1.2731,  1.2731,  ..., -0.8978, -1.0203, -1.6506],
          [ 1.2906,  1.2906,  1.2731,  ..., -0.8978, -1.0028, -1.5280],
          ...,
          [ 0.3452,  0.3277,  0.2927,  ...,  0.3978,  0.4328,  0.4678],
          [ 0.3452,  0.3277,  0.3102,  ...,  0.4153,  0.4503,  0.4678],
          [ 0.3277,  0.3102,  0.2927,  ...,  0.3978,  0.4503,  0.4678]],

         [[ 1.3851,  1.3502,  1.3328,  ..., -0.7936, -0.8807, -1.4733],
          [ 1.4025,  1.

...

Data:  tensor([[[[ 1.4440,  1.4098,  1.3927,  ...,  1.8550,  1.8379,  1.8208],
          [ 1.1872,  1.1529,  1.1187,  ...,  1.9064,  1.8550,  1.8037],
          [ 1.0673,  1.0502,  1.0502,  ...,  1.9064,  1.8379,  1.7865],
          ...,
          [ 0.0569,  0.0912,  0.2111,  ..., -0.5596, -0.4911, -0.4054],
          [-0.0287,  0.0227,  0.2111,  ..., -0.5424, -0.4397, -0.4226],
          [-0.1143, -0.0116,  0.0912,  ..., -0.5596, -0.4739, -0.4226]],

         [[ 1.8683,  1.8508,  1.7983,  ...,  1.8683,  1.8333,  1.8333],
          [ 1.5532,  1.5182,  1.4482,  ...,  1.8683,  1.8158,  1.8158],
          [ 1.3431,  1.3431,  1.3431,  ...,  1.8508,  1.8333,  1.8158],
          ...,
          [-0.1625, -0.1275,  0.0301,  ..., -0.8803, -0.8452, -0.8102],
          [-0.2150, -0.1275,  0.0651,  ..., -0.8978, -0.7927, -0.8102],
          [-0.3200, -0.2675, -0.1099,  ..., -0.9328, -0.8102, -0.7927]],

         [[ 2.0648,  2.0474,  2.0648,  ...,  1.9254,  1.8905,  1.8905],
          [ 1.6988,  1.

Data:  tensor([[[[ 0.0056,  0.1939,  0.1939,  ...,  0.4337,  0.2453,  0.7077],
          [ 0.1083,  0.1597,  0.6734,  ...,  0.5022,  0.2967,  0.1768],
          [ 0.3481,  0.4851,  0.5536,  ...,  0.7762,  0.7077,  0.3309],
          ...,
          [-1.2788, -1.1589, -1.0390,  ..., -0.5082, -0.4397, -0.4397],
          [-1.2617, -1.1589, -1.0904,  ..., -0.2171, -0.4226, -0.4226],
          [-1.3130, -1.0219, -1.0904,  ..., -0.0801, -0.4911, -0.7137]],

         [[ 0.4853,  0.7304,  0.7654,  ...,  1.0805,  1.0455,  1.4657],
          [ 0.6254,  0.7129,  1.1506,  ...,  1.0805,  0.9930,  0.9055],
          [ 0.9405,  1.1331,  1.1856,  ...,  1.3782,  1.3606,  1.1155],
          ...,
          [-1.1604, -1.0903, -0.9678,  ...,  0.0126,  0.0651, -0.1099],
          [-1.1429, -1.0728, -0.9853,  ...,  0.1702, -0.0749, -0.1275],
          [-1.2304, -1.0028, -1.0728,  ...,  0.2752, -0.2325, -0.3901]],

         [[-0.6367, -0.3404, -0.4450,  ..., -0.7761, -0.8633, -0.8110],
          [-0.4275, -0.

Data:  tensor([[[[ 2.1975,  2.2318,  2.2147,  ..., -0.7479, -0.7822, -0.7479],
          [ 2.1633,  2.1975,  2.1975,  ..., -0.7650, -0.7650, -0.6794],
          [ 2.1633,  2.1975,  2.2147,  ..., -0.6965, -0.8507, -0.8164],
          ...,
          [ 1.6838,  1.7180,  1.7523,  ...,  1.8550,  1.8208,  1.8379],
          [ 1.6667,  1.7694,  1.8037,  ...,  1.8893,  1.8379,  1.8550],
          [ 1.7180,  1.7523,  1.7694,  ...,  1.8722,  1.8893,  1.9064]],

         [[ 2.2360,  2.3060,  2.3410,  ..., -1.2829, -1.3179, -1.2829],
          [ 2.2535,  2.3060,  2.3410,  ..., -1.3354, -1.3880, -1.3529],
          [ 2.2535,  2.3235,  2.3410,  ..., -1.3529, -1.3704, -1.3529],
          ...,
          [ 1.9209,  1.9209,  1.9209,  ...,  2.0784,  2.0609,  2.0784],
          [ 1.9384,  1.9384,  1.9559,  ...,  2.0959,  2.0784,  2.0609],
          [ 1.9034,  1.8859,  1.9384,  ...,  2.0784,  2.0959,  2.1134]],

         [[ 2.3960,  2.4483,  2.5180,  ..., -1.3513, -1.3861, -1.3861],
          [ 2.3960,  2.

Data:  tensor([[[[-0.3369, -0.3369, -0.3369,  ..., -0.4911, -0.4568, -0.1999],
          [-0.3883, -0.3712, -0.4226,  ..., -1.3815, -1.2445, -0.9363],
          [-0.4568, -0.4226, -0.5082,  ..., -1.3815, -1.2617, -1.1932],
          ...,
          [-1.6042, -1.1760, -1.3644,  ..., -1.4500, -1.3644, -1.3130],
          [-1.4329, -0.8335, -0.9705,  ..., -1.4843, -1.3644, -1.2959],
          [-1.1075, -0.8164, -0.6452,  ..., -1.4672, -1.3473, -1.2959]],

         [[ 0.2227,  0.1702,  0.1001,  ..., -0.8102, -0.7927, -0.8277],
          [ 0.1527,  0.0476, -0.0574,  ..., -1.2829, -1.2129, -1.1429],
          [ 0.0301, -0.1625, -0.3025,  ..., -1.4405, -1.4580, -1.4580],
          ...,
          [-1.1954, -0.7227, -0.8102,  ..., -0.7577, -0.8102, -0.8452],
          [-0.9853, -0.3725, -0.3901,  ..., -0.8277, -0.8452, -0.8102],
          [-0.5476, -0.2500, -0.0224,  ..., -0.8627, -0.8277, -0.8277]],

         [[-0.2881, -0.5321, -0.8807,  ..., -0.5147, -0.4624, -0.2707],
          [-0.3055, -0.

Data:  tensor([[[[-0.9020, -0.9020, -0.8507,  ..., -1.0219, -1.0733, -1.0733],
          [-0.9877, -0.9363, -0.8849,  ..., -1.1589, -1.2445, -1.0904],
          [-0.9363, -0.9192, -0.8507,  ..., -1.1075, -1.0048, -0.9705],
          ...,
          [-0.8849, -0.7822, -0.8507,  ..., -1.0048, -1.1418, -1.4500],
          [-0.8678, -1.1589, -1.2274,  ..., -0.8849, -1.3987, -1.5185],
          [-1.1589, -1.2445, -1.1075,  ..., -1.0562, -1.6042, -1.3130]],

         [[-0.5476, -0.5651, -0.5126,  ..., -0.6527, -0.7052, -0.7052],
          [-0.6176, -0.5651, -0.5126,  ..., -0.7927, -0.8803, -0.7227],
          [-0.6176, -0.6001, -0.5476,  ..., -0.7402, -0.6352, -0.6001],
          ...,
          [-0.6176, -0.5126, -0.5826,  ..., -0.7402, -0.8803, -1.1954],
          [-0.6001, -0.8978, -0.9678,  ..., -0.6176, -1.1429, -1.2654],
          [-0.8978, -0.9853, -0.8277,  ..., -0.7927, -1.3529, -1.0553]],

         [[-1.2293, -1.2293, -1.1944,  ..., -1.4036, -1.4559, -1.4559],
          [-1.2816, -1.

In [1]:
import os
import numpy as np
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

## Define transforms, Datasets, batch_size, and Dataloaders

## While I have used various combinations of augmentation, I eventualy removed the RandomRotations and RandomResizedCrops
## after implementing batch normalization, which aocording to the paper introducing the concept, relieves the need
## for these kinds of augmentation. 
## There was no difference in training results whether using the additional augementation measures or not.

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.Resize(250),
                                      transforms.RandomCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])
                                     ])

test_transform = transforms.Compose([transforms.Resize(250),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], 
                                                          [0.229, 0.224, 0.225])
                                    ])


data_dir = 'dogImages/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

print('Num Training imgs: ', len(train_data))
print('Num Validation imgs: ', len(valid_data))
print('Num Test imgs: ', len(test_data))

batch_size = 8
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)


loaders_scratch = {'train': train_loader, 
                   'valid': valid_loader, 
                   'test': test_loader}

classes = train_data.classes

Num Training imgs:  6680
Num Validation imgs:  835
Num Test imgs:  836


In [ ]:
print(classes)

Note: Should these photos really be normalized with these mean and standard deviation values if it distorts their color this way? With this normalization, pixel values go well beyond the -1, 1 range that I'd think would be optimal.

Normalizing with mean = [0.5, 0.5, 0.5] and std= [0.5, 0.5, 0.5] results in images that look much more natural - and with pixel values that are actually in the range from -1 to 0. 

I have tried both ways, and my network has failed to train either way, but just a thought...

## Architecture 1: 
* 5 Conv Layers: 32 to 512 channels - all with 3x3 kernels, stride=1 and padding=1
* Batch Normalization and ReLU activation on each Convolutional Layer
* MaxPool(2,2) after each Convolutional Layer
* 3 fully connected layers with ReLU applied to fc1 and fc2
* Dropout with probability of 50% applied after 2nd, 3rd, and 4th convolutional layer (before maxpool layers) and after 1st and 2nd fc layer

* CrossEntropyLoss (applied as log_softmax on 3rd fc layer and NLLLoss as criterion
* Adam optimizer with lr=0.0025
* Result: after 10 training epochs:

Epoch: 1 	Training Loss: 5.363417 	Validation Loss: 4.876212

Epoch: 2 	Training Loss: 4.888124 	Validation Loss: 4.870528

Epoch: 3 	Training Loss: 4.909029 	Validation Loss: 4.868946

Epoch: 4 	Training Loss: 4.878525 	Validation Loss: 4.867552

Epoch: 5 	Training Loss: 4.878072 	Validation Loss: 4.867843

Epoch: 6 	Training Loss: 4.872267 	Validation Loss: 4.868251

Epoch: 7 	Training Loss: 4.866718 	Validation Loss: 4.868318

Epoch: 8 	Training Loss: 4.867238 	Validation Loss: 4.867780

Epoch: 9 	Training Loss: 4.866567 	Validation Loss: 4.868664

Epoch: 10 	Training Loss: 4.866593 	Validation Loss: 4.868259

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512*7*7, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 500, bias=True)
        self.fc3 = nn.Linear(500, 133, bias=True)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm4(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm5(self.conv5(x)))
        x = self.pool(x)
        
        x = x.view(-1, 512*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

In [5]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.NLLLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.0025)

In [ ]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)  #output is tensor of shape([batch_size, num_classes]) where largest value is the prediction
            loss = criterion(output, target) # loss is the cross-entropy loss which measures how far the prediction is from the actual target
            loss.backward()  # calculating the gradients for all operations
            optimizer.step() #performing gradient descent step
            train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

# train the model
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

## Architecture 2:
* Same convolutional layers, batch norm, activation, and max pool layers as above
* only 2 fully connected layers 
* dropout at .3
* changed loss function to actual nn.CrossEntropyLoss (and removed non-linearity from last fully connected layer)
* learning rate increased to .025
* Same training/validation loop as above

### Results after 10 training epochs:
Epoch: 1 	Training Loss: 20.446863 	Validation Loss: 4.880278

Epoch: 2 	Training Loss: 4.894538 	Validation Loss: 4.889751

Epoch: 3 	Training Loss: 4.894520 	Validation Loss: 4.891614

Epoch: 4 	Training Loss: 4.894344 	Validation Loss: 4.887182

Epoch: 5 	Training Loss: 4.925039 	Validation Loss: 4.884497

Epoch: 6 	Training Loss: 4.895196 	Validation Loss: 4.884095

Epoch: 7 	Training Loss: 4.901975 	Validation Loss: 4.882802

Epoch: 8 	Training Loss: 4.895335 	Validation Loss: 4.886300

Epoch: 9 	Training Loss: 4.894879 	Validation Loss: 4.884101

Epoch: 10 	Training Loss: 4.894126 	Validation Loss: 4.884886


In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512*7*7, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 133, bias=True)
#         self.fc3 = nn.Linear(500, 133, bias=True)
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm4(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm5(self.conv5(x)))
        x = self.pool(x)
        
        x = x.view(-1, 512*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

In [8]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.025)

In [9]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data) #get predictions
            loss = criterion(output, target) #calulate loss
            loss.backward()  # calculate the gradients
            optimizer.step() # perform optimization step
            train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

# train the model
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

Epoch: 1 	Training Loss: 20.446863 	Validation Loss: 4.880278
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.894538 	Validation Loss: 4.889751
Epoch: 3 	Training Loss: 4.894520 	Validation Loss: 4.891614
Epoch: 4 	Training Loss: 4.894344 	Validation Loss: 4.887182
Epoch: 5 	Training Loss: 4.925039 	Validation Loss: 4.884497
Epoch: 6 	Training Loss: 4.895196 	Validation Loss: 4.884095
Epoch: 7 	Training Loss: 4.901975 	Validation Loss: 4.882802
Epoch: 8 	Training Loss: 4.895335 	Validation Loss: 4.886300
Epoch: 9 	Training Loss: 4.894879 	Validation Loss: 4.884101
Epoch: 10 	Training Loss: 4.894126 	Validation Loss: 4.884886


The fact that I consistently get one epoch where the training loss shows obvious progress followed by a bunch of minor shifts back and forth seems to indicate a problem with my training loop - as though the gradients get turned off in the validation loop, and are never turned back on. I will run one more version only changing my architecture, then will run each of these three models again with a change to my training code (that seems like it should be unnecessary) to ensure that the gradients are turned back on during each training loop.

## Architecture 3:
* Batch Normalization switched to apply After ReLU on each convolutional layer
* all else remains the same as above

### Result after 10 training epochs:
Epoch: 1 	Training Loss: 22.869417 	Validation Loss: 4.881092

Epoch: 2 	Training Loss: 4.894224 	Validation Loss: 4.884323

Epoch: 3 	Training Loss: 4.912950 	Validation Loss: 5.126093

Epoch: 4 	Training Loss: 4.897206 	Validation Loss: 4.928959

Epoch: 5 	Training Loss: 4.895872 	Validation Loss: 5.007126

Epoch: 6 	Training Loss: 4.892983 	Validation Loss: 5.116531

Epoch: 7 	Training Loss: 4.893521 	Validation Loss: 5.319191

Epoch: 8 	Training Loss: 4.897356 	Validation Loss: 4.882479

Epoch: 9 	Training Loss: 4.894560 	Validation Loss: 4.885508

Epoch: 10 	Training Loss: 4.894811 	Validation Loss: 4.889231


In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512*7*7, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 133, bias=True)
#         self.fc3 = nn.Linear(500, 133, bias=True)
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.pool(x)
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.pool(x)
        
        x = x.view(-1, 512*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

In [4]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.025)

In [5]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data) #get predictions
            loss = criterion(output, target) #calulate loss
            loss.backward()  # calculate the gradients
            optimizer.step() # perform optimization step
            train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

# train the model
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

Epoch: 1 	Training Loss: 22.869417 	Validation Loss: 4.881092
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.894224 	Validation Loss: 4.884323
Epoch: 3 	Training Loss: 4.912950 	Validation Loss: 5.126093
Epoch: 4 	Training Loss: 4.897206 	Validation Loss: 4.928959
Epoch: 5 	Training Loss: 4.895872 	Validation Loss: 5.007126
Epoch: 6 	Training Loss: 4.892983 	Validation Loss: 5.116531
Epoch: 7 	Training Loss: 4.893521 	Validation Loss: 5.319191
Epoch: 8 	Training Loss: 4.897356 	Validation Loss: 4.882479
Epoch: 9 	Training Loss: 4.894560 	Validation Loss: 4.885508
Epoch: 10 	Training Loss: 4.894811 	Validation Loss: 4.889231


## Architecture 3 with modified training loop:
* training loop modifeid to place all training steps inside `with torch.enable_grad():` loop
* learning rate decreased to 0.0025 to closer approximate what has been reported as optimal
* directly accessing train_loader and valid_loader, rather than accessing through the loaders dict.
* iterating through train_loader and valid_loader directly, rather than enumerating them. (I don't use the batch index for any reason anyway)

### Results after 10 training epochs:
Epoch: 1 	Training Loss: 6.145714 	Validation Loss: 317.693266

Epoch: 2 	Training Loss: 4.881616 	Validation Loss: 149.638213

Epoch: 3 	Training Loss: 4.881738 	Validation Loss: 76.856270

Epoch: 4 	Training Loss: 4.868647 	Validation Loss: 10.119001

Epoch: 5 	Training Loss: 4.870256 	Validation Loss: 175.708004

Epoch: 6 	Training Loss: 4.885781 	Validation Loss: 5.842024

Epoch: 7 	Training Loss: 4.867848 	Validation Loss: 5.502650

Epoch: 8 	Training Loss: 4.866408 	Validation Loss: 5.315064

Epoch: 9 	Training Loss: 4.866485 	Validation Loss: 5.140727

Epoch: 10 	Training Loss: 4.866341 	Validation Loss: 5.599622


It's at least showing something different than the versions before it - only in the validation loops however. 
The training loss shows the same as before: 1 good step, followed by bouncing around in the 4.8 range.

The validation loss showed progress for a few iterations, before exploding in Epoch 5, then dropping back down to the 5.x level in epoch 6 after which it just bounced around in the 5.x range without progress.

I will keep this checkpoint saved in the model_scratch.pt file and will rename the checkpoint for the next attempts.

Testing shows accuracy of 1%. Predictions seem to start as random, then later settle as class 4 for every image.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512*7*7, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 133, bias=True)
#         self.fc3 = nn.Linear(500, 133, bias=True)
        self.dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.pool(x)
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.pool(x)
        
        x = x.view(-1, 512*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

In [3]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.025)

In [4]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            with torch.enable_grad():
                optimizer.zero_grad()
                output = model(data) #get predictions
                loss = criterion(output, target) #calulate loss
                loss.backward()  # calculate the gradients
                optimizer.step() # perform optimization step
                train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for data, target in valid_loader:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

# train the model
model_scratch = train(30, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch_3.pt')

Epoch: 1 	Training Loss: 21.230738 	Validation Loss: 95.244353
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.893078 	Validation Loss: 24.413463
Validation Loss Decreased. Saving model
Epoch: 3 	Training Loss: 4.896302 	Validation Loss: 1088.203040
Epoch: 4 	Training Loss: 4.894740 	Validation Loss: 33.911446
Epoch: 5 	Training Loss: 4.895293 	Validation Loss: 1834.588041
Epoch: 6 	Training Loss: 4.897745 	Validation Loss: 1198.642593
Epoch: 7 	Training Loss: 4.896015 	Validation Loss: 1604.381039
Epoch: 8 	Training Loss: 4.894294 	Validation Loss: 3534.684728
Epoch: 9 	Training Loss: 4.893817 	Validation Loss: 2232.242461
Epoch: 10 	Training Loss: 4.893525 	Validation Loss: 2071.671996
Epoch: 11 	Training Loss: 4.894349 	Validation Loss: 1978.222531
Epoch: 12 	Training Loss: 4.896489 	Validation Loss: 1439.038064
Epoch: 13 	Training Loss: 4.894585 	Validation Loss: 2151.704347
Epoch: 14 	Training Loss: 4.895108 	Validation Loss: 152.453516
Epoch: 15 	Training Loss:

## Architecture 1 again, with modified training loop:
* Using same training/validation code as that used with Architecture 1, but adding the `torch.enable_grad():` line to encompass the training loop
* also changed the average loss calculation prior to printing from using `train_loss/len(train_loader)` to using `train_loss/len(loaders['train'])`, which I believe should be insignificant.

### Results after 10 epochs
Epoch: 1 	Training Loss: 5.625305 	Validation Loss: 4.871825

Epoch: 2 	Training Loss: 4.876564 	Validation Loss: 4.869084

Epoch: 3 	Training Loss: 4.872355 	Validation Loss: 4.871287

Epoch: 4 	Training Loss: 4.870470 	Validation Loss: 4.868716

Epoch: 5 	Training Loss: 4.869439 	Validation Loss: 4.870064

Epoch: 6 	Training Loss: 4.867907 	Validation Loss: 4.868820

Epoch: 7 	Training Loss: 4.867843 	Validation Loss: 4.867972

Epoch: 8 	Training Loss: 4.867246 	Validation Loss: 4.868295

Epoch: 9 	Training Loss: 4.866786 	Validation Loss: 4.868938

Epoch: 10 	Training Loss: 4.878639 	Validation Loss: 4.867290

No better than any other. 1 seemingly good step, followed by bouncing around in 4.8 range as usual.
Test shows accuracy of 1% - with every single image receiving a prediction of class 4.

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512*7*7, 1000, bias=True)
        self.fc2 = nn.Linear(1000, 500, bias=True)
        self.fc3 = nn.Linear(500, 133, bias=True)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm4(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(x)
        x = F.relu(self.batchnorm5(self.conv5(x)))
        x = self.pool(x)
        
        x = x.view(-1, 512*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

In [6]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.NLLLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.0025)

In [7]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            with torch.enable_grad():
                optimizer.zero_grad()
                output = model(data)  #output is tensor of shape([batch_size, num_classes]) where largest value is the prediction
                loss = criterion(output, target) # loss is the cross-entropy loss which measures how far the prediction is from the actual target
                loss.backward()  # calculating the gradients for all operations
                optimizer.step() #performing gradient descent step
                train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(loaders['train'])
        valid_loss = valid_loss / len(loaders['valid'])
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

# train the model
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch_2.pt')

Epoch: 1 	Training Loss: 5.625305 	Validation Loss: 4.871825
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.876564 	Validation Loss: 4.869084
Validation Loss Decreased. Saving model
Epoch: 3 	Training Loss: 4.872355 	Validation Loss: 4.871287
Epoch: 4 	Training Loss: 4.870470 	Validation Loss: 4.868716
Validation Loss Decreased. Saving model
Epoch: 5 	Training Loss: 4.869439 	Validation Loss: 4.870064
Epoch: 6 	Training Loss: 4.867907 	Validation Loss: 4.868820
Epoch: 7 	Training Loss: 4.867843 	Validation Loss: 4.867972
Validation Loss Decreased. Saving model
Epoch: 8 	Training Loss: 4.867246 	Validation Loss: 4.868295
Epoch: 9 	Training Loss: 4.866786 	Validation Loss: 4.868938
Epoch: 10 	Training Loss: 4.878639 	Validation Loss: 4.867290
Validation Loss Decreased. Saving model


## Test function

In [6]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.0
    correct = 0.0
    total = 0.0
    model.eval()
    
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            print('Data: ', data)
            print('target: ', target)
            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Data:  tensor([[[[-1.4672, -1.4329, -1.3644,  ..., -1.4843, -1.5014, -1.5185],
          [-1.5014, -1.4329, -1.3130,  ..., -1.4672, -1.4843, -1.4843],
          [-1.5185, -1.4500, -1.3302,  ..., -1.4329, -1.4500, -1.4843],
          ...,
          [ 2.0777,  2.0777,  2.0605,  ..., -1.3815, -1.3644, -1.3473],
          [ 2.0777,  2.0605,  2.0605,  ..., -1.3473, -1.3815, -1.3815],
          [ 2.0777,  2.0434,  2.0434,  ..., -1.3815, -1.4158, -1.3987]],

         [[-0.9853, -0.9503, -0.8978,  ..., -0.9678, -0.9853, -1.0028],
          [-1.0028, -0.9328, -0.8452,  ..., -0.9853, -0.9678, -0.9678],
          [-0.9853, -0.9153, -0.8277,  ..., -0.9678, -0.9853, -0.9853],
          ...,
          [ 1.4657,  1.4657,  1.4482,  ..., -1.0378, -1.0203, -1.0378],
          [ 1.4657,  1.4307,  1.4307,  ..., -0.9853, -1.0028, -1.0203],
          [ 1.4482,  1.4132,  1.4132,  ..., -1.0203, -1.0378, -1.0378]],

         [[-1.4907, -1.5081, -1.4733,  ..., -1.2990, -1.3164, -1.3339],
          [-1.5256, -1.

Data:  tensor([[[[-1.3815, -1.2788, -1.3302,  ...,  1.0844,  0.3994, -0.1999],
          [-1.3987, -1.4158, -1.3473,  ...,  1.0159,  0.9817,  0.3994],
          [-0.8507, -1.5014, -1.5014,  ...,  0.8618,  0.8276,  0.4679],
          ...,
          [ 1.3755,  0.7762, -0.5938,  ...,  0.0569,  0.8618,  1.3584],
          [ 1.4783,  0.9303, -0.0629,  ..., -1.5185, -1.1247, -0.1828],
          [ 1.6838,  0.7933,  0.5364,  ..., -1.3987, -0.9534, -0.0972]],

         [[-0.7752, -0.6352, -0.7927,  ...,  1.3081,  0.4678, -0.2150],
          [-0.9328, -0.9503, -0.9503,  ...,  1.3081,  1.2031,  0.5903],
          [-0.4776, -1.1429, -1.1253,  ...,  1.0980,  1.1856,  0.8179],
          ...,
          [ 0.9405,  0.3627, -1.0203,  ..., -0.5301,  0.2402,  0.6954],
          [ 1.0455,  0.4678, -0.4951,  ..., -1.6681, -1.4580, -0.5826],
          [ 1.3256,  0.4153,  0.1001,  ..., -1.5455, -1.1429, -0.3725]],

         [[-1.3861, -1.3164, -1.4210,  ...,  0.4265, -0.1835, -0.8110],
          [-1.2641, -1.

Data:  tensor([[[[-0.9705, -0.9534, -0.9534,  ...,  1.0331,  0.9988,  1.0502],
          [-1.0219, -1.0219, -1.0390,  ...,  0.9474,  0.9132,  0.9646],
          [-0.9192, -0.9363, -0.9705,  ...,  0.8961,  0.8789,  0.9132],
          ...,
          [ 1.3755,  1.3927,  1.3927,  ..., -0.9020, -0.7822, -0.8507],
          [ 1.3755,  1.3927,  1.3927,  ..., -0.8507, -0.7650, -0.9363],
          [ 1.3755,  1.3927,  1.3927,  ..., -0.7822, -0.7993, -1.0390]],

         [[-0.6877, -0.6702, -0.6702,  ...,  0.1702,  0.1527,  0.2052],
          [-0.7402, -0.7402, -0.7577,  ...,  0.0826,  0.0476,  0.1001],
          [-0.6352, -0.6527, -0.6877,  ...,  0.0301,  0.0126,  0.0301],
          ...,
          [ 1.5357,  1.5532,  1.5532,  ..., -0.9678, -0.8627, -0.9678],
          [ 1.5357,  1.5532,  1.5532,  ..., -0.9503, -0.8627, -1.0553],
          [ 1.5357,  1.5532,  1.5532,  ..., -0.8803, -0.9153, -1.1604]],

         [[-0.3055, -0.2881, -0.2881,  ..., -0.8458, -0.8807, -0.8458],
          [-0.3404, -0.

Data:  tensor([[[[ 1.6324,  0.0056,  0.0056,  ..., -1.0733, -0.9877, -1.0048],
          [ 1.5639, -0.0801, -0.0801,  ..., -1.0904, -1.0390, -1.0048],
          [ 1.2899, -0.0801, -0.0287,  ..., -1.0904, -1.0562, -1.0048],
          ...,
          [-0.9534, -0.7308, -0.9363,  ..., -1.5528, -1.0733, -0.9705],
          [-0.7993, -0.8507, -0.7137,  ..., -1.2274, -1.1418, -1.2445],
          [-0.5767, -0.9363, -0.7479,  ..., -1.2445, -1.2274, -1.2959]],

         [[ 1.9384,  0.2402,  0.1527,  ..., -0.2850, -0.1975, -0.1975],
          [ 1.8859,  0.1877,  0.0651,  ..., -0.2850, -0.2325, -0.1975],
          [ 1.6057,  0.1877,  0.1352,  ..., -0.2850, -0.2500, -0.1975],
          ...,
          [-0.5126, -0.3200, -0.4251,  ..., -1.3704, -0.8627, -0.7052],
          [-0.1975, -0.3725, -0.3200,  ..., -1.0203, -0.8627, -0.8978],
          [ 0.1352, -0.4251, -0.4251,  ..., -1.0028, -0.8452, -0.8627]],

         [[ 1.8557,  0.0779,  0.2522,  ...,  0.9494,  1.0539,  1.0539],
          [ 1.7685,  0.

       device='cuda:0')
Output:  tensor([[-4.9118e-01, -3.6031e-01, -1.3381e-01,  ..., -1.0861e+00,
         -1.1508e+00, -1.2135e+00],
        [-4.9118e-01, -3.6031e-01, -1.3381e-01,  ..., -1.0861e+00,
         -1.1508e+00, -1.2135e+00],
        [-4.9118e-01, -3.6031e-01, -1.3381e-01,  ..., -1.0861e+00,
         -1.1508e+00, -1.2135e+00],
        ...,
        [-1.4882e+04, -1.0445e+04, -1.5580e+04,  ..., -1.2266e+04,
         -1.1713e+04, -1.5692e+04],
        [-1.1380e+05, -6.4257e+04, -1.1777e+05,  ..., -8.9054e+04,
         -9.6022e+04, -1.1970e+05],
        [-3.3350e+01, -3.1345e+01, -3.3692e+01,  ...,  7.1913e+00,
         -2.4132e+01, -4.0143e+01]], device='cuda:0')
target:  tensor([122,  91,  71,  37,   5, 110,  58,  18], device='cuda:0')
tensor([[ 4],
        [ 4],
        [ 4],
        [56],
        [ 4],
        [56],
        [56],
        [47]], device='cuda:0')
Data:  tensor([[[[-0.7479, -0.7479, -0.7308,  ...,  0.5878,  0.7248,  0.7762],
          [-0.7479, -0.7650, -0.78

...

Data:  tensor([[[[ 0.0227,  0.0569,  0.1083,  ..., -0.2513, -0.2513, -0.2171],
          [-0.0116,  0.0741,  0.1254,  ..., -0.2856, -0.2342, -0.2856],
          [-0.0287,  0.0398,  0.0569,  ..., -0.3027, -0.3027, -0.3369],
          ...,
          [ 0.4337,  0.3994,  0.1254,  ...,  0.5022,  0.4679,  0.5536],
          [ 1.1358,  0.8104,  0.2796,  ...,  0.5878,  0.3823,  0.7591],
          [ 1.0673,  1.0502,  0.8789,  ...,  0.7419,  0.4851,  0.5536]],

         [[ 0.0476,  0.0826,  0.1001,  ...,  0.0126,  0.0126,  0.0476],
          [ 0.0301,  0.0826,  0.1176,  ..., -0.0224,  0.0301, -0.0224],
          [ 0.0126,  0.0826,  0.1001,  ..., -0.0399, -0.0399, -0.0749],
          ...,
          [ 0.4328,  0.4328,  0.2227,  ...,  0.8179,  0.8179,  0.9405],
          [ 1.0980,  0.8004,  0.3627,  ...,  0.9055,  0.7304,  1.1331],
          [ 0.9755,  1.0280,  0.9230,  ...,  1.0455,  0.8354,  0.9580]],

         [[-1.2293, -1.1944, -1.1596,  ..., -1.1073, -1.1073, -1.0724],
          [-1.2467, -1.

Data:  tensor([[[[-0.4397, -0.3712, -0.2856,  ...,  1.7523,  1.8550,  1.9407],
          [-0.2684, -0.1828, -0.1314,  ...,  1.7523,  1.8550,  1.9407],
          [-0.0801,  0.0227,  0.0569,  ...,  1.7523,  1.8550,  1.9407],
          ...,
          [ 0.3652,  0.3481,  0.3481,  ...,  0.4679,  0.5022,  0.5022],
          [ 0.3309,  0.3138,  0.3309,  ...,  0.4679,  0.4851,  0.4851],
          [ 0.3138,  0.2967,  0.2967,  ...,  0.4679,  0.4679,  0.4679]],

         [[-1.0378, -1.0203, -0.9853,  ...,  1.2381,  1.2906,  1.3256],
          [-0.8803, -0.8627, -0.8277,  ...,  1.2381,  1.2906,  1.3256],
          [-0.7052, -0.6527, -0.6527,  ...,  1.2381,  1.2906,  1.3256],
          ...,
          [ 1.0455,  1.0280,  1.0280,  ...,  1.0980,  1.1331,  1.1331],
          [ 1.0105,  0.9930,  1.0105,  ...,  1.0980,  1.1155,  1.1155],
          [ 0.9930,  0.9755,  0.9755,  ...,  1.0980,  1.0980,  1.0980]],

         [[-0.9330, -0.9504, -0.8981,  ...,  0.5485,  0.3742,  0.1476],
          [-0.8110, -0.

Output:  tensor([[ -0.4912,  -0.3603,  -0.1338,  ...,  -1.0861,  -1.1508,  -1.2135],
        [-60.7477, -13.9784, -65.0344,  ..., -57.5465, -68.1473, -65.7572],
        [ -0.4912,  -0.3603,  -0.1338,  ...,  -1.0861,  -1.1508,  -1.2135],
        ...,
        [ -0.4912,  -0.3603,  -0.1338,  ...,  -1.0861,  -1.1508,  -1.2135],
        [-69.4640,  -5.4692, -71.6080,  ..., -63.0048, -70.2237, -59.9931],
        [-20.8908,  15.2818, -24.6035,  ..., -19.7211, -25.1101, -20.7410]],
       device='cuda:0')
target:  tensor([ 28,   0,  86,  43,  78, 108,   1, 109], device='cuda:0')
tensor([[  4],
        [ 31],
        [  4],
        [ 47],
        [ 36],
        [  4],
        [106],
        [ 47]], device='cuda:0')
Data:  tensor([[[[ 1.2728,  1.3070,  1.1529,  ...,  0.7419,  0.7248,  0.7248],
          [ 1.2385,  1.2557,  1.1187,  ...,  0.7762,  0.7591,  0.7762],
          [ 1.2043,  1.1872,  1.1015,  ...,  0.9132,  0.8447,  0.8276],
          ...,
          [-0.2171, -0.4739, -0.1999,  ...,  0

target:  tensor([ 32, 123,  35,  82,  72, 111,  63,  81], device='cuda:0')
tensor([[ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [36],
        [ 4],
        [ 4]], device='cuda:0')
Data:  tensor([[[[ 1.1700,  1.3413,  1.1187,  ...,  0.4851,  0.7762,  0.7762],
          [ 1.2557,  1.2385,  1.0331,  ...,  0.3823,  0.5707,  0.5878],
          [ 1.0502,  1.2385,  0.8789,  ..., -0.0458,  0.1768,  0.3994],
          ...,
          [-0.1143, -0.2171, -0.0458,  ..., -0.2856, -0.4397, -0.6109],
          [-0.0458,  0.0056,  0.0227,  ..., -0.4397, -0.3883, -0.3883],
          [-0.2513, -0.0972, -0.0287,  ..., -0.6965, -0.5424, -0.3541]],

         [[ 1.5882,  1.7458,  1.5007,  ...,  0.8354,  1.2206,  1.2906],
          [ 1.6583,  1.6408,  1.3957,  ...,  0.7129,  1.0980,  1.2731],
          [ 1.4482,  1.6057,  1.2381,  ...,  0.3102,  0.7304,  1.1155],
          ...,
          [ 0.3978,  0.2577,  0.3803,  ...,  0.3803,  0.0826, -0.2325],
          [ 0.3803,  0.3978,  0.3803

         -0.6527, -0.5687, -1.0861, -1.1508, -1.2135]], device='cuda:0')
target:  tensor([103,  43, 132,   6], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4]], device='cuda:0')
Test Loss: 1345.552979


Test Accuracy:  0% ( 7/836)


### Running architecture 1 again, but directly accessing global variables inside of train function

#### Result:
Again, predicting the same class for every image (class 4 - seems to be a favorite)

In [6]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    global train_loader
    global valid_loader
    global model_scratch
    global optimizer_scratch
    global criterion_scratch
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model_scratch.train()
        for data, target in train_loader:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            with torch.enable_grad():
                optimizer_scratch.zero_grad()
                output = model_scratch(data) #get predictions
                loss = criterion_scratch(output, target) #calulate loss
                loss.backward()  # calculate the gradients
                optimizer_scratch.step() # perform optimization step
                train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model_scratch.eval()
        for data, target in valid_loader:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model_scratch(data)
                loss = criterion_scratch(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model_scratch.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model_scratch

# train the model
trained_model = train(20, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'arch_1_global_train_variables.pt')

Epoch: 1 	Training Loss: 5.593780 	Validation Loss: 4.872661
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.880664 	Validation Loss: 4.870785
Validation Loss Decreased. Saving model
Epoch: 3 	Training Loss: 4.872185 	Validation Loss: 4.868692
Validation Loss Decreased. Saving model
Epoch: 4 	Training Loss: 4.870217 	Validation Loss: 4.868035
Validation Loss Decreased. Saving model
Epoch: 5 	Training Loss: 4.868786 	Validation Loss: 4.868854
Epoch: 6 	Training Loss: 4.867970 	Validation Loss: 4.868792
Epoch: 7 	Training Loss: 4.887500 	Validation Loss: 4.871854
Epoch: 8 	Training Loss: 4.867568 	Validation Loss: 4.868095
Epoch: 9 	Training Loss: 4.866680 	Validation Loss: 4.869761
Epoch: 10 	Training Loss: 4.866748 	Validation Loss: 4.869246
Epoch: 11 	Training Loss: 4.866648 	Validation Loss: 4.868374
Epoch: 12 	Training Loss: 4.866472 	Validation Loss: 4.868917
Epoch: 13 	Training Loss: 4.866339 	Validation Loss: 4.867692
Validation Loss Decreased. Saving model
Epo

### Test function accessing global variables, rather than parameters

In [9]:
def test(loaders, model, criterion, use_cuda):
    global test_loader
    global model_scratch
    global trained_model
    global criterion_scratch
    
    # monitor test loss and accuracy
    test_loss = 0.0
    correct = 0.0
    total = 0.0
    model_scratch.eval()
    
    for data, target in test_loader:
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            # forward pass: compute predicted outputs by passing inputs to the model
            output = trained_model(data)
            print('Output: ', output)
            print('target: ', target)
            # calculate the loss
            loss = criterion_scratch(output, target)
            # update average test loss 
#             test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            test_loss += loss.item()
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss/len(test_loader)))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([127,  59,   7,  48,  26,  13, 106,  51], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([28, 91, 46, 30, 43, 89, 26, 74], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., 

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([105,  21,  56,  96,  11,  38, 119,  16], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([ 71,  40,  34, 122,  17,  42,  47,  79], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([122, 117,  26,  96,  25,  72,   1,  13], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([15, 93, 67, 51, 67, 57, 23, 87], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., 

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([ 47,  21,   8,  16,  54, 103,  72, 102], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548

Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034]],
       device='cuda:0')
target:  tensor([ 62,  19,  40, 118, 108,  56,   1,  63], device='cuda:0')
tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]], device='cuda:0')
Output:  tensor([[-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        ...,
        [-4.6683, -4.7427, -4.8548,  ..., -5.4137, -5.5457, -5.4034],
        [-4.6683, -4.7427, -4.8548

## Test

In [8]:
# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

<All keys matched successfully>

In [9]:
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

Data:  tensor([[[[ 0.0912,  0.1254,  0.1939,  ..., -0.0972, -0.0629, -0.3198],
          [ 0.1426,  0.1597,  0.2111,  ..., -0.1657, -0.0801, -0.3369],
          [ 0.1083,  0.1254,  0.1768,  ..., -0.0801,  0.0056, -0.1657],
          ...,
          [-0.9705, -0.8678, -0.6452,  ..., -1.1589, -1.0904, -1.0219],
          [-1.0733, -0.9705, -0.7822,  ..., -1.1247, -1.0904, -1.0562],
          [-0.9020, -0.7650, -0.7308,  ..., -1.0733, -1.0904, -1.0904]],

         [[ 1.0105,  1.0280,  1.0280,  ...,  0.3102,  0.3978,  0.2227],
          [ 1.1155,  1.1331,  1.1331,  ...,  0.2927,  0.4503,  0.2752],
          [ 1.0455,  1.0630,  1.0980,  ...,  0.3627,  0.5378,  0.4328],
          ...,
          [ 0.0126,  0.0301,  0.2227,  ..., -0.1099, -0.0574, -0.0049],
          [-0.1099, -0.0749,  0.0651,  ..., -0.0749, -0.0749, -0.0574],
          [-0.0574,  0.0126,  0.0126,  ..., -0.0399, -0.0749, -0.1099]],

         [[ 0.2522,  0.2696,  0.2871,  ..., -0.3578, -0.2707, -0.4450],
          [ 0.3219,  0.

Data:  tensor([[[[-0.9192, -0.7822, -0.5767,  ...,  0.2796,  0.2453,  0.1426],
          [-0.9020, -0.7822, -0.6794,  ...,  0.2282,  0.1939,  0.1939],
          [-0.9192, -0.7822, -0.7137,  ...,  0.1939,  0.2624,  0.2796],
          ...,
          [-0.4397, -0.5767, -0.8507,  ..., -0.4911, -0.3712, -0.4568],
          [-0.4054, -0.5253, -0.9192,  ..., -0.3541, -0.5767, -0.2513],
          [-0.4397, -0.2342, -0.7137,  ..., -0.2513, -0.2856, -0.1828]],

         [[-0.9153, -0.7927, -0.5651,  ...,  0.3452,  0.2577,  0.1877],
          [-0.8978, -0.7577, -0.6352,  ...,  0.2577,  0.2052,  0.2052],
          [-0.8627, -0.7227, -0.6527,  ...,  0.2402,  0.2927,  0.3102],
          ...,
          [-0.4601, -0.5826, -0.8803,  ..., -0.5126, -0.4076, -0.4776],
          [-0.4426, -0.5301, -0.9328,  ..., -0.3375, -0.6001, -0.2500],
          [-0.4601, -0.2325, -0.7227,  ..., -0.2325, -0.2675, -0.1450]],

         [[-0.9678, -0.8458, -0.6193,  ...,  0.3916,  0.3393,  0.2696],
          [-0.9504, -0.

Data:  tensor([[[[ 1.5297,  1.6324,  1.7523,  ...,  1.2214, -0.1143, -0.7308],
          [ 1.4612,  1.6838,  1.6153,  ...,  0.9303, -0.3712, -0.7479],
          [ 1.5468,  1.6153,  1.3242,  ...,  0.9474, -0.3712, -0.7308],
          ...,
          [ 0.1768,  0.1939, -0.2513,  ...,  0.8789,  1.1187,  1.3070],
          [-0.0458, -0.0458, -0.1314,  ...,  0.1939,  0.0056,  0.4166],
          [-0.1657, -0.1143,  0.0398,  ...,  0.5536,  0.1254, -0.2171]],

         [[ 1.9384,  2.0434,  2.1485,  ...,  1.5007,  0.1702, -0.3901],
          [ 1.8333,  2.0609,  1.9909,  ...,  1.2031, -0.0924, -0.4601],
          [ 1.9209,  1.9909,  1.6933,  ...,  1.2381, -0.0749, -0.4251],
          ...,
          [ 0.2752,  0.2927, -0.1625,  ...,  0.6429,  0.9055,  1.1506],
          [ 0.0476,  0.0476, -0.0399,  ..., -0.0049, -0.1625,  0.2927],
          [-0.0749, -0.0224,  0.1352,  ...,  0.4328,  0.0301, -0.2850]],

         [[ 2.4134,  2.4657,  2.5180,  ...,  2.0997,  0.7925,  0.2173],
          [ 2.3437,  2.

Data:  tensor([[[[ 0.9474,  1.0159,  1.0331,  ..., -1.6384, -1.5699, -1.6042],
          [ 0.9646,  1.0844,  1.0159,  ..., -1.6042, -1.5185, -1.5528],
          [ 0.9988,  1.1358,  1.0331,  ..., -1.5870, -1.5699, -1.5699],
          ...,
          [ 2.2489,  2.2489,  2.2489,  ...,  2.1119,  1.9235,  2.1290],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2318,  2.2318,  2.2147],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2147,  2.2318,  2.2318]],

         [[ 1.1155,  1.2731,  1.3431,  ..., -1.4230, -1.4405, -1.4405],
          [ 1.1331,  1.3256,  1.3431,  ..., -1.4230, -1.4405, -1.4755],
          [ 1.1506,  1.3606,  1.3606,  ..., -1.4755, -1.4755, -1.4580],
          ...,
          [ 2.4286,  2.4286,  2.4286,  ...,  2.2010,  1.9034,  2.2360],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4111,  2.4111],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4111,  2.4111,  2.4111]],

         [[ 1.2282,  1.5420,  1.6291,  ..., -1.1944, -1.1770, -1.1421],
          [ 1.2457,  1.

Data:  tensor([[[[-0.5424, -0.5767, -0.5938,  ..., -1.1932, -1.2103, -1.0904],
          [-0.4911, -0.5424, -0.5767,  ..., -1.1589, -1.1932, -1.0562],
          [-0.4054, -0.4568, -0.4739,  ..., -1.1760, -1.1760, -1.0562],
          ...,
          [ 0.8276,  0.7762,  0.6734,  ...,  0.3481,  0.3138,  0.2967],
          [ 0.6563,  0.6563,  0.6221,  ...,  0.3481,  0.3481,  0.3309],
          [ 0.3994,  0.4337,  0.4337,  ...,  0.2453,  0.2796,  0.3138]],

         [[-1.0028, -1.0378, -1.0203,  ..., -1.7381, -1.8081, -1.7206],
          [-0.9503, -1.0028, -1.0028,  ..., -1.7031, -1.7906, -1.6856],
          [-0.8627, -0.9153, -0.8978,  ..., -1.7031, -1.7556, -1.6856],
          ...,
          [ 1.0630,  1.0455,  0.9580,  ...,  0.4678,  0.4328,  0.4153],
          [ 0.8704,  0.8704,  0.8704,  ...,  0.4853,  0.4678,  0.4678],
          [ 0.6078,  0.6429,  0.6604,  ...,  0.3978,  0.4328,  0.4503]],

         [[-0.9330, -0.9330, -0.8807,  ..., -1.6650, -1.7347, -1.6824],
          [-0.8807, -0.

...

Data:  tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          ...,
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2489,  2.2489,  2.2489]],

         [[ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          ...,
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286],
          [ 2.4286,  2.4286,  2.4286,  ...,  2.4286,  2.4286,  2.4286]],

         [[ 2.6400,  2.6400,  2.6400,  ...,  2.6400,  2.6400,  2.6400],
          [ 2.6400,  2.

       device='cuda:0')
Output:  tensor([[ 2.0585e-01,  9.1996e-02, -1.4494e-02,  ..., -5.4787e-01,
         -6.6454e-01, -5.4926e-01],
        [ 2.0585e-01,  9.1996e-02, -1.4494e-02,  ..., -5.4787e-01,
         -6.6454e-01, -5.4926e-01],
        [ 1.9628e-01,  9.4907e-02,  8.5020e-04,  ..., -6.7567e-01,
         -6.1368e-01, -6.0342e-01],
        ...,
        [ 2.0585e-01,  9.1996e-02, -1.4494e-02,  ..., -5.4787e-01,
         -6.6454e-01, -5.4926e-01],
        [ 2.0585e-01,  9.1996e-02, -1.4494e-02,  ..., -5.4787e-01,
         -6.6454e-01, -5.4926e-01],
        [-9.7186e-01, -2.5791e+00, -1.4065e+00,  ..., -3.1536e+00,
         -2.7706e+00, -1.3946e+00]], device='cuda:0')
target:  tensor([ 11,  66, 100,  72,  40,  48,  41,  67], device='cuda:0')
tensor([[ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [48]], device='cuda:0')
Data:  tensor([[[[-0.2342, -0.2342, -0.2513,  ..., -0.3712, -0.3712, -0.3369],
          [-0.1999, -0.2171, -0.23

Data:  tensor([[[[ 1.3584,  1.4098,  1.4269,  ..., -0.0116, -0.0116, -0.0287],
          [ 1.3242,  1.3584,  1.3927,  ..., -0.6794, -0.6623, -0.6452],
          [ 1.3413,  1.3755,  1.3584,  ..., -0.7822, -0.7137, -0.6623],
          ...,
          [-1.3815, -1.3302, -1.2788,  ...,  0.4851,  0.5364,  0.5364],
          [-1.3644, -1.3130, -1.2959,  ...,  0.4851,  0.5364,  0.5536],
          [-1.3130, -1.2788, -1.2959,  ...,  0.5193,  0.5364,  0.5193]],

         [[ 1.3782,  1.4307,  1.4657,  ...,  0.1527,  0.1527,  0.1352],
          [ 1.3431,  1.3606,  1.3606,  ..., -0.4076, -0.3725, -0.3550],
          [ 1.3256,  1.3606,  1.3431,  ..., -0.5651, -0.5126, -0.4251],
          ...,
          [-1.4405, -1.4230, -1.3704,  ...,  0.4678,  0.5028,  0.5028],
          [-1.4230, -1.3880, -1.3880,  ...,  0.4678,  0.5028,  0.5203],
          [-1.3704, -1.3529, -1.3880,  ...,  0.4853,  0.5028,  0.4853]],

         [[ 1.5071,  1.5594,  1.5768,  ...,  0.1476,  0.1476,  0.1651],
          [ 1.4374,  1.

Data:  tensor([[[[-1.7069, -1.8268, -1.6727,  ..., -1.2788, -1.5699, -1.6727],
          [-1.8782, -1.9638, -1.7069,  ..., -0.6623, -0.2342, -1.1247],
          [-1.8439, -1.8439, -1.7240,  ..., -1.2959, -0.2513, -0.4226],
          ...,
          [-1.0562, -1.0390, -1.1418,  ..., -0.0458, -0.1828,  0.0398],
          [-0.7650, -1.0219, -0.9705,  ...,  0.4166, -0.1999, -0.1657],
          [-0.8849, -1.0048, -0.6794,  ...,  0.8447,  0.3652, -0.0972]],

         [[-1.5105, -1.6155, -1.4755,  ..., -0.7577, -1.0903, -1.1954],
          [-1.6681, -1.7556, -1.4930,  ..., -0.1275,  0.3102, -0.6001],
          [-1.6331, -1.6331, -1.5105,  ..., -0.7927,  0.3102,  0.1527],
          ...,
          [-0.4601, -0.3901, -0.4601,  ...,  0.7129,  0.5728,  0.8004],
          [-0.1625, -0.3725, -0.3025,  ...,  1.1856,  0.5553,  0.5903],
          [-0.2850, -0.3725, -0.0224,  ...,  1.6232,  1.1331,  0.6604]],

         [[-1.5430, -1.6302, -1.4907,  ..., -1.5081, -1.5256, -1.5081],
          [-1.6999, -1.

Data:  tensor([[[[ 0.5364,  0.6221,  0.5364,  ...,  0.5022,  0.5536,  0.5364],
          [ 0.5707,  0.4679,  0.6906,  ...,  0.5022,  0.5536,  0.5022],
          [ 0.5536,  0.5707,  0.7591,  ...,  0.4851,  0.5707,  0.6049],
          ...,
          [ 0.7077,  0.7419,  0.7591,  ...,  0.5536,  0.5022,  0.5707],
          [ 0.7591,  0.7077,  0.6906,  ...,  0.5364,  0.4679,  0.5364],
          [ 0.7762,  0.7077,  0.6734,  ...,  0.4679,  0.4337,  0.4679]],

         [[ 0.6779,  0.7654,  0.6779,  ...,  0.6254,  0.6779,  0.6604],
          [ 0.7129,  0.6078,  0.8354,  ...,  0.6254,  0.6779,  0.6254],
          [ 0.6954,  0.7129,  0.8880,  ...,  0.6078,  0.6954,  0.7304],
          ...,
          [ 0.8354,  0.8704,  0.8880,  ...,  0.6604,  0.6078,  0.6779],
          [ 0.8880,  0.8354,  0.8179,  ...,  0.6429,  0.5728,  0.6429],
          [ 0.9055,  0.8354,  0.8004,  ...,  0.5728,  0.5378,  0.5728]],

         [[ 0.8971,  0.9842,  0.8971,  ...,  0.8099,  0.8622,  0.8448],
          [ 0.9319,  0.

### Ignore the code below this line
It's just a bunch of random pieces of code I've used at various times while trying to figure out what was going on.

In [ ]:
# # Visualize the data

# import matplotlib.pyplot as plt
# %matplotlib inline

# dataiter = iter(train_loader)
# images, labels = dataiter.next()
# images = images.numpy()
# fig = plt.figure(figsize=(25,4))
# for idx in np.arange(6):
#     ax = fig.add_subplot(2, 6/2, idx+1, xticks=[], yticks=[])
#     images[idx] = images[idx]/2 +0.5
#     plt.imshow(np.transpose(images[idx], (1,2,0)))
#     ax.set_title(classes[labels[idx]])
    

In [8]:
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# valid_loss_min = np.Inf

# epochs = 5
# for e in range(epochs):
#     running_loss = 0.0
#     for images, labels in train_loader:
#         images, labels = images.cuda(), labels.cuda()
#         optimizer_scratch.zero_grad()
#         output = model_scratch(images)
#         loss = criterion_scratch(output, labels)
#         loss.backward()
#         optimizer_scratch.step()
#         running_loss += loss.item()
#     else:
#         print("Training loss: {:.6f}".format(running_loss/len(train_loader)))



Training loss: 9.782534
Training loss: 5.036802
Training loss: 4.944024
Training loss: 4.900999
Training loss: 5.088080


In [13]:

# import matplotlib.pyplot as plt
# %matplotlib inline
# images, labels = next(iter(train_loader))
# images, labels = images.cuda(), labels.cuda()
# with torch.no_grad():
#     logps = model_scratch(images)
# probs = torch.exp(logps)


In [15]:
# print(images[0])

tensor([[[ 0.6000,  0.6000,  0.6078,  ...,  0.5843,  0.5686,  0.5608],
         [ 0.6078,  0.6078,  0.6078,  ...,  0.5765,  0.5608,  0.5608],
         [ 0.6235,  0.6157,  0.6078,  ...,  0.5686,  0.5608,  0.5765],
         ...,
         [-0.0980, -0.2392, -0.4196,  ...,  0.0824, -0.3333, -0.4118],
         [-0.1216, -0.5294, -0.5059,  ...,  0.0667, -0.3020, -0.5608],
         [-0.0588,  0.0275, -0.0275,  ..., -0.0118, -0.2314,  0.2706]],

        [[ 0.6078,  0.6078,  0.6078,  ...,  0.5843,  0.5686,  0.5608],
         [ 0.6157,  0.6157,  0.6078,  ...,  0.5765,  0.5608,  0.5608],
         [ 0.6235,  0.6235,  0.6078,  ...,  0.5686,  0.5608,  0.5765],
         ...,
         [-0.1294, -0.2392, -0.3490,  ...,  0.1216, -0.3098, -0.3961],
         [-0.1451, -0.5294, -0.4588,  ...,  0.0980, -0.2941, -0.5608],
         [-0.0824,  0.0275,  0.0196,  ..., -0.0039, -0.2314,  0.2627]],

        [[ 0.6549,  0.6627,  0.6863,  ...,  0.6784,  0.6627,  0.6549],
         [ 0.6549,  0.6627,  0.6863,  ...,  0

In [19]:
# print(images[0][0])
# print(images[0][1])
# print(images[1][0])

tensor([[ 0.6000,  0.6000,  0.6078,  ...,  0.5843,  0.5686,  0.5608],
        [ 0.6078,  0.6078,  0.6078,  ...,  0.5765,  0.5608,  0.5608],
        [ 0.6235,  0.6157,  0.6078,  ...,  0.5686,  0.5608,  0.5765],
        ...,
        [-0.0980, -0.2392, -0.4196,  ...,  0.0824, -0.3333, -0.4118],
        [-0.1216, -0.5294, -0.5059,  ...,  0.0667, -0.3020, -0.5608],
        [-0.0588,  0.0275, -0.0275,  ..., -0.0118, -0.2314,  0.2706]],
       device='cuda:0')
tensor([[ 0.6078,  0.6078,  0.6078,  ...,  0.5843,  0.5686,  0.5608],
        [ 0.6157,  0.6157,  0.6078,  ...,  0.5765,  0.5608,  0.5608],
        [ 0.6235,  0.6235,  0.6078,  ...,  0.5686,  0.5608,  0.5765],
        ...,
        [-0.1294, -0.2392, -0.3490,  ...,  0.1216, -0.3098, -0.3961],
        [-0.1451, -0.5294, -0.4588,  ...,  0.0980, -0.2941, -0.5608],
        [-0.0824,  0.0275,  0.0196,  ..., -0.0039, -0.2314,  0.2627]],
       device='cuda:0')
tensor([[-0.0353, -0.0196,  0.0039,  ..., -0.7961, -0.7882, -0.7882],
        [-0.03

In [6]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):

        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)  #output is tensor of shape([batch_size, num_classes]) where largest value is the prediction
            loss = criterion(output, target) # loss is the cross-entropy loss which measures how far the prediction is from the actual target
            loss.backward()  # calculating the gradients for all operations
            optimizer.step() #performing gradient descent step
            train_loss += loss.item()

    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()
            
        train_loss = train_loss / len(train_loader)
        valid_loss = valid_loss / len(valid_loader)
        
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation Loss Decreased. Saving model')
            torch.save(model.state_dict(), 'model_scratch.pt')
            valid_loss_min = valid_loss
    # return trained model
    return model



In [7]:
# train the model
model_scratch = train(10, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')



Epoch: 1 	Training Loss: 5.363417 	Validation Loss: 4.876212
True
Validation Loss Decreased. Saving model
Epoch: 2 	Training Loss: 4.888124 	Validation Loss: 4.870528
True
Validation Loss Decreased. Saving model
Epoch: 3 	Training Loss: 4.909029 	Validation Loss: 4.868946
True
Validation Loss Decreased. Saving model
Epoch: 4 	Training Loss: 4.878525 	Validation Loss: 4.867552
True
Validation Loss Decreased. Saving model
Epoch: 5 	Training Loss: 4.878072 	Validation Loss: 4.867843
True
Epoch: 6 	Training Loss: 4.872267 	Validation Loss: 4.868251
True
Epoch: 7 	Training Loss: 4.866718 	Validation Loss: 4.868318
True
Epoch: 8 	Training Loss: 4.867238 	Validation Loss: 4.867780
True
Epoch: 9 	Training Loss: 4.866567 	Validation Loss: 4.868664
True
Epoch: 10 	Training Loss: 4.866593 	Validation Loss: 4.868259
True


In [ ]:
# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

In [ ]:
def test(model, )
    for batch_idx, (data, target) in enumerate(test_loader):
        if use_cuda():
            data, target = data.cuda(), target.cuda()
        output = model_scratch(data)

In [9]:


# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

Epoch: 1 	Training Loss: 8.555840 	Validation Loss: 4.883201
Validation Loss Decreased. Saving model


KeyboardInterrupt: 